In [1]:
# you will be prompted with a window asking to grant permissions
# click connect to google drive, choose your account, and click allow
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
# TODO: fill in the path in your Google Drive in the string below
# Note: do not escape slashes or spaces in the path string
import os
os.chdir("/content/drive/MyDrive/cs444-project_belavise") # fill in with the path to the google drive folder where your mp is.

In [3]:
# reload imports when running a function(this will make it so you don't have to reload the entire notebook if you change a function that you are importing and want the change to register)
%reload_ext autoreload
%autoreload 2

In [4]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.8 MB/s eta 0:00:00


In [19]:
os.mkdir('data')
os.chdir('data')

In [20]:
!wget https://saurabhg.web.illinois.edu/teaching/cs444/fa2023/mp3/coco.zip -O coco.zip
!unzip coco.zip

Streaming output truncated to the last 5000 lines.
  inflating: coco/train2017/000000252846.jpg  
  inflating: coco/train2017/000000196664.jpg  
  inflating: coco/train2017/000000057992.jpg  
  inflating: coco/train2017/000000028820.jpg  
  inflating: coco/train2017/000000377422.jpg  
  inflating: coco/train2017/000000099543.jpg  
  inflating: coco/train2017/000000467318.jpg  
  inflating: coco/train2017/000000178971.jpg  
  inflating: coco/train2017/000000477679.jpg  
  inflating: coco/train2017/000000564611.jpg  
  inflating: coco/train2017/000000574402.jpg  
  inflating: coco/train2017/000000160858.jpg  
  inflating: coco/train2017/000000073830.jpg  
  inflating: coco/train2017/000000098107.jpg  
  inflating: coco/train2017/000000052168.jpg  
  inflating: coco/train2017/000000568765.jpg  
  inflating: coco/train2017/000000561027.jpg  
  inflating: coco/train2017/000000391825.jpg  
  inflating: coco/train2017/000000276066.jpg  
  inflating: coco/train2017/000000225358.jpg  
  inflati

In [ ]:
#### Preprocess Cococ data

In [22]:
os.chdir('../')

In [23]:
import pickle


from pycocotools.coco import COCO
from torchtext.vocab import GloVe

In [24]:
from transformers import BertTokenizer, BertModel

In [25]:
import torch

In [26]:
def compute_cat_mappings_2(ann_file: str, bert_model: BertModel, bert_tokenizer: BertTokenizer) -> dict:
    """
    Compute mappings from COCO categories to BERT embeddings.

    Args:
        ann_file: Path to the annotation file.
        bert_model: BERT model.
        bert_tokenizer: BERT tokenizer.

    Returns:
        dict: A dictionary containing the mappings. The keys are:
            - "embeddings" (dict): COCO category to BERT embedding.
            - "itos" (dict): BERT embedding index to COCO category.
    """
    print("Computing category mappings...", end=" ")
    coco = COCO(ann_file)
    cat_mappings = {"stoi": {}, "itos": {}}

    for cat in coco.cats.values():
        # Tokenize the category name
        tokens = bert_tokenizer.tokenize(cat["name"])
        # Convert tokens to BERT input IDs
        input_ids = bert_tokenizer.convert_tokens_to_ids(tokens)

        # Get BERT embeddings for the input IDs
        #with torch.no_grad():
        #    embeddings = bert_model(torch.tensor([input_ids]))[0].mean(dim=1).squeeze().tolist()
        print(input_ids)
        # Assign the embeddings to the category
        cat_mappings["stoi"][cat["name"]] = input_ids[0]
        cat_mappings["itos"][input_ids[0]] = cat["name"]

    print("Done.")
    return cat_mappings

In [ ]:
def compute_cat_mappings(ann_file: str, glove: GloVe) \
        -> dict[str, dict]:
    """
    Compute mappings from COCO categories to GloVe indices.

    Args:
        ann_file: Path to the annotation file.
        glove: GloVe object.

    Returns:
        dict: A dictionary containing the mappings. The keys are:
            - "stoi" (dict): COCO category to GloVe index.
            - "itos" (dict): GloVe index to COCO category.
    """
    print("Computing category mappings...", end=" ")
    coco = COCO(ann_file)
    cat_mappings = {"stoi": {}, "itos": {}}
    for cat in coco.cats.values():
        for token in cat["name"].split(" "):
            cat_mappings["stoi"][token] = glove.stoi[token]
            cat_mappings["itos"][glove.stoi[token]] = token
    print("Done.")
    return cat_mappings

In [27]:
root = "./data/coco"
#glove = GloVe(name="6B", dim=300)
bert_model = BertModel.from_pretrained("bert-base-uncased")
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Compute the category mappings.
ann_file = os.path.join(root, "annotations/instances_train2017-animal-nocrowd.json")
cat_mappings = compute_cat_mappings_2(ann_file, bert_model, bert_tokenizer)
with open(os.path.join(root, "cat_mappings_bert.pkl"), "wb") as f:
    pickle.dump(cat_mappings, f)

Computing category mappings... loading annotations into memory...
Done (t=0.35s)
creating index...
index created!
[4743]
[4937]
[3899]
[3586]
[8351]
[11190]
[10777]
[4562]
[29145]
[21025, 27528, 7959]
Done.


In [28]:
!python train_explainer.py --model resnet18 --layer-target layer4 --batch-size 16 --name resnet18_bert_25 --layer-classifier fc --refer coco --epochs 25

args=Namespace(batch_size=16, cat_frac=0.7, dir_data='./data', dir_save='./outputs', epochs=25, layer_classifier='fc', layer_target='layer4', margin=1.0, model='resnet18', name='resnet18_bert_25', num_workers=8, path_model=None, random=False, refer='coco', save_every=1000, seed=42, train_frac=0.9, wandb=False, word_embedding_dim=768)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100% 44.7M/44.7M [00:00<00:00, 89.4MB/s]
No model found at './outputs/resnet18_bert_25/ckpt_best.pth.tar'. Training from scratch.
loading annotations into memory...
Done (t=0.43s)
creating index...
index created!
loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
Processing BERT embeddings:  4562
Processing BERT embeddings:  4743
Processing BERT embeddings:  4937
Processing BERT embeddings:  11190
Processing BERT embeddings:  3899
Processing BERT embeddings:  10777
Processing BERT embeddings:  

In [30]:
!python infer_filter.py --model resnet18 --layer-target layer4 --layer-classifier fc --refer coco --name resnet18_bert_25 --path-model "outputs/resnet18_bert_25/ckpt_best.pth.tar" --s 20

args=Namespace(batch_size=1, dir_data='./data', dir_save='./outputs', layer_classifier='fc', layer_target='layer4', method='projection', model='resnet18', name='resnet18_bert_25', num_heatmaps=5, num_tokens=10, num_workers=8, p=25, path_model='outputs/resnet18_bert_25/ckpt_best.pth.tar', random=False, refer='coco', s=20, seed=42, threshold_iou=0.04, u=[0, 12, 24, 36, 48, 60, 72, 84, 96, 108, 120, 132, 144, 156, 168, 180, 192, 204, 216, 228, 240, 252], wandb=False, weigh_s_by_relevance=False, word_embedding_dim=768)
Loading model from 'outputs/resnet18_bert_25/ckpt_best.pth.tar'... Done.
loading annotations into memory...
Done (t=0.23s)
creating index...
index created!
loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this Dat